# Bagging Machine Learning Algorithm

### **B**ootstrap **Agg**regat**ing** or [Bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating)
* [Scikit- Learn Reference](http://scikit-learn.org/stable/modules/ensemble.html#bagging)
* Bootstrap sampling: Sampling with replacement
* Combine by averaging the output (regression)
* Combine by voting (classification)
* Can be applied to many classifiers which includes ANN, CART, etc.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [ ]:
df = sns.load_dataset('titanic')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['pclass'].unique()

In [ ]:
df['pclass'].value_counts()

In [ ]:
df['sex'].unique()

In [ ]:
df['sex'].value_counts()

In [ ]:
df['age'].hist(bins=50);

## Data Pre-processing

In [ ]:
X = df[['pclass', 'sex', 'age']]

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

In [ ]:
X['sex'] = lb.fit_transform(X['sex'])

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
X.info()

In [ ]:
y = df['survived']

In [ ]:
y.value_counts()

# Fit Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Importing Evaluation Metrics

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Defining a function so that we can get all the evaluation metrics at one place for each run

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    '''
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))

## Decision Tree

In [ ]:
clf = DecisionTreeClassifier(random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print_score(clf, X_train, y_train, X_test, y_test, train=True)

In [ ]:
print_score(clf, X_train, y_train, X_test, y_test, train=False) # Test

***

## Bagging (oob_score=False)

In [ ]:
bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=5000,
                            bootstrap=True, n_jobs=-1,
                            random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)

In [ ]:
print_score(bag_clf, X_train, y_train, X_test, y_test, train=True)

In [ ]:
print_score(bag_clf, X_train, y_train, X_test, y_test, train=False)

***

## Bagging (oob_score=True)

Use out-of-bag samples to estimate the generalization accuracy

In [ ]:
bag_clf = BaggingClassifier(base_estimator=clf, n_estimators=1000,
                            bootstrap=True, oob_score=True,
                            n_jobs=-1, random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)

In [ ]:
bag_clf.oob_score_

In [ ]:
print_score(bag_clf, X_train, y_train, X_test, y_test, train=True)

In [ ]:
print_score(bag_clf, X_train, y_train, X_test, y_test, train=False)

# Exercise: 

# Now try executing the same with n_estimators = 5000 and oob_score = True / False

***